<a href="https://colab.research.google.com/github/chryssa-pat/anakthsh/blob/main/ColBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3.10-venv
!pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate
import torch
print(torch.cuda.is_available())

!pip install git+https://github.com/stanford-futuredata/ColBERT.git
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10-venv is already the newest version (3.10.12-1~22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
created virtual environment CPython3.10.12.final.0-64 in 1162ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
True
  Cloning https://github.com/stanford-futuredata/ColBERT.git to /tmp/pip-req-build-x_t_pxvs
  Running command git clone --filter=blob:none --quiet https://github.com/stanford-futuredata/ColBERT.git /tmp/pip-req-build-x_t_pxvs
  Resolved https://github.com/stanford-futuredata/ColBERT.git to c

In [ ]:
import google.colab
!pip install -U pip
!pip install -e ColBERT/['faiss-gpu','torch']

Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert-ai
    Found existing installation: colbert-ai 0.2.17
    Uninstalling colbert-ai-0.2.17:
      Successfully uninstalled colbert-ai-0.2.17
  Running setup.py develop for colbert-ai


In [ ]:
# Import necessary components from colbert library
import colbert
from colbert.modeling.colbert import ColBERT
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection


In [ ]:
try:
    from colbert import Indexer, Searcher
    from colbert.infra import Run, RunConfig, ColBERTConfig
    from colbert.data import Queries, Collection

    print("ColBERT is imported successfully!")
except ImportError as e:
    print(f"Error importing ColBERT: {e}")

ColBERT is imported successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ir/docs.zip -d /content

Archive:  /content/drive/MyDrive/ir/docs.zip
  inflating: /content/docs/00001     
  inflating: /content/docs/00002     
  inflating: /content/docs/00003     
  inflating: /content/docs/00004     
  inflating: /content/docs/00005     
  inflating: /content/docs/00006     
  inflating: /content/docs/00007     
  inflating: /content/docs/00008     
  inflating: /content/docs/00009     
  inflating: /content/docs/00010     
  inflating: /content/docs/00011     
  inflating: /content/docs/00012     
  inflating: /content/docs/00013     
  inflating: /content/docs/00014     
  inflating: /content/docs/00015     
  inflating: /content/docs/00016     
  inflating: /content/docs/00017     
  inflating: /content/docs/00018     
  inflating: /content/docs/00019     
  inflating: /content/docs/00020     
  inflating: /content/docs/00021     
  inflating: /content/docs/00022     
  inflating: /content/docs/00023     
  inflating: /content/docs/00024     
  inflating: /content/docs/00025     
  inf

In [ ]:
import os
import csv

def read_documents(directory_path):
    documents = []
    for file in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file)
        with open(file_path, 'r', encoding="utf8") as folder:
            text = folder.read()
            doc_id = file
            document_tuple = (doc_id, text)
            documents.append(document_tuple)
    return documents



# Set the path to your documents and queries
path = '/content/docs'
queries_path = '/content/Queries_20'

# Read documents
documents = read_documents(path)

def read_queries(file_path):
    queries = []
    with open(file_path, 'r', encoding="utf8") as file:
        text = file.read()
        queries_list = text.split('\n')
        queries_list = [query.strip() for query in queries_list if query.strip()]
        queries.extend(queries_list)
    return queries
# Read queries
queries = read_queries(queries_path)


# Set up ColBERT configurations
nbits = 2
doc_maxlen = 300
max_id = 1209
index_name = 'index'
checkpoint = '/content/drive/MyDrive/ir/colbertv2.0'

# Indexing
with Run().context(RunConfig(nranks=1, experiment='indexing')):
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=[doc[1] for doc in documents[:max_id]], overwrite=True)

# Searching
with Run().context(RunConfig(experiment='indexing')):
    searcher = Searcher(index=index_name, collection=[doc[1] for doc in documents])


# Create a dictionary to store lists of document IDs for each query
colbert_result = {}


# Iterate over all queries
for query in queries:
    print(f"#> {query}")
    results = searcher.search(query, k=1209)  # Retrieve all passages
    result_list = []
    for i, (passage_id, passage_rank, passage_score) in enumerate(zip(*results)):
       doc_id = documents[int(passage_id)][0]
       print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t  Document ID: {doc_id}")
       result_list.append(doc_id)
    colbert_result[query] = result_list




# Display or save the dictionary
print(colbert_result)

# Save the dictionary to a CSV file
csv_file_path = '/content/colbert_result.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(['Query', 'DocumentIDs'])

    # Write the data
    for query, document_list in colbert_result.items():
        writer.writerow([query, document_list])

print(f"Query document lists saved to {csv_file_path}")



[Jan 22, 11:33:51] #> Note: Output directory /content/experiments/indexing/indexes/index already exists


[Jan 22, 11:33:51] #> Will delete 10 files already at /content/experiments/indexing/indexes/index in 20 seconds...
#> Starting...
#> Joined...
[Jan 22, 11:34:39] #> Loading codec...
[Jan 22, 11:34:39] #> Loading IVF...
[Jan 22, 11:34:40] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1294.14it/s]

[Jan 22, 11:34:40] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 207.33it/s]

#> What are the effects of calcium on the physical properties of mucus from CF patients

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What are the effects of calcium on the physical properties of mucus from CF patients, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2024,  1996,  3896,  1997, 13853,  2006,  1996,
         3558,  5144,  1997, 14163,  7874,  2013, 12935,  5022,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

	 [1] 		 23.1 		  Document ID: 00533
	 [2] 		 20.4 		  Document ID: 00441
	 [3] 		 19.4 		  Document ID: 00957
	 [4] 		 19.3 		  Document ID: 00484
	 [5] 		 18.6 		  Document ID: 00741
	 [6] 		 18.3 		  Document ID: 00827
	 [7] 		 18.3 		  Document I

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
	 [694] 		 10.9 		  Document ID: 00182
	 [695] 		 10.9 		  Document ID: 00965
	 [696] 		 10.9 		  Document ID: 00177
	 [697] 		 10.9 		  Document ID: 01072
	 [698] 		 10.9 		  Document ID: 00126
	 [699] 		 10.9 		  Document ID: 01195
	 [700] 		 10.9 		  Document ID: 00252
	 [701] 		 10.9 		  Document ID: 00053
	 [702] 		 10.9 		  Document ID: 00453
	 [703] 		 10.8 		  Document ID: 00426
	 [704] 		 10.8 		  Document ID: 00006
	 [705] 		 10.8 		  Document ID: 00327
	 [706] 		 10.8 		  Document ID: 01000
	 [707] 		 10.8 		  Document ID: 00865
	 [708] 		 10.8 		  Document ID: 00806
	 [709] 		 10.8 		  Document ID: 00644
	 [710] 		 10.8 		  Document ID: 00650
	 [711] 		 10.8 		  Document ID: 01129
	 [712] 		 10.8 		  Document ID: 00415
	 [713] 		 10.8 		  Document ID: 01180
	 [714] 		 10.8 		  Document ID: 00047
	 [715] 		 10.8 		  Document ID: 01055
	 [716] 		 10.8 		  Document ID: 00816
	 [717] 		 10.8 		  Document ID: 00983
	 [718] 	